In [37]:
# Dependence
import uuid 
import cv2
import os
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
from gtts import gTTS  
from pygame import mixer

In [38]:
# initialize mediapipe
mpHands = mp.solutions.hands

In [39]:
# initialize Hands from mediapipe

# config
maxNumHands = 1
minDetectionConfidence=0.7

hands = mpHands.Hands(
    max_num_hands=maxNumHands,
    min_detection_confidence=minDetectionConfidence
    )

In [40]:
# initialize drawing_utils from mediapipe 
mpDraw = mp.solutions.drawing_utils

In [41]:
# Load the gesture recognizer model
model = load_model('Modal')

In [42]:
# Load class names
f = open('gesture.names', 'r')

# converting list into array of strings
classNames = f.read().split('\n')
f.close()

In [43]:
print('Hands gestures list')
print(classNames)

# Hands gestures list
# ['okay', 'thank you', 'Yes', 'No', 'call me', 'stop', 'Love you', 'live long', 'fist', 'end']

Hands gestures list
['okay', 'thank you', 'Yes', 'No', 'call me', 'stop', 'Love you', 'live long', 'fist', 'end']


In [44]:
# Initialize the webcam
cap = cv2.VideoCapture(0)

In [45]:
# hand gesture recognition to speech.
inProgress= True
last_action = None
while inProgress:
    # Read each nframe from the webcam
    _, frame = cap.read()

    x, y, c = frame.shape

    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get hand landmark prediction
    result = hands.process(framergb)

    # print(result)
    
    className = ''
    
    # xd boost,adda bosst
    # add features
    # post process the result
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                # print(id, lm)
                lmx = int(lm.x * x)
                lmy = int(lm.y * y)

                landmarks.append([lmx, lmy])

            # Drawing landmarks on frames
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

            # Predict gesture
            prediction = model.predict([landmarks])
            # print(prediction)
            classID = np.argmax(prediction)
            className = classNames[classID]
            temp =''

            isSame = className == last_action
            last_action = className
            
            if not isSame:
                print(className)
            
            
            if className == 'stop':
                inProgress=False

            language = 'en'  
            if className != temp and not isSame: 
                id = uuid.uuid4()
                temp = className
                obj = gTTS(text=temp, lang=language, slow=False)  
    
                #Here we are saving the transformed audio in a mp3 file named  
                # exam.mp3  
                try:
                    obj.save(f'temp/{id}.mp3')  
                except Exception:
                    print(Exception)

                mixer.init()
                mixer.music.load(f'D:/1111/major project/temp/{id}.mp3')
                mixer.music.play()
            
    # show the prediction on the frame
    cv2.putText(frame, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                   1, (0,0,255), 2, cv2.LINE_AA)

   
    
    # Show the final output
    cv2.imshow("Output", frame) 

    if cv2.waitKey(1) == ord('q'):
        break

1/1 [==============================] - 0s 141ms/step
live long
1/1 [==============================] - 0s 53ms/step
Love you
1/1 [==============================] - 0s 58ms/step
fist
1/1 [==============================] - 0s 62ms/step
Love you
1/1 [==============================] - 0s 53ms/step
fist
1/1 [==============================] - 0s 50ms/step
Love you
1/1 [==============================] - 0s 49ms/step
live long
1/1 [==============================] - 0s 46ms/step
stop


In [46]:
# release the webcam and destroy all active windows
cap.release()

cv2.destroyAllWindows()

In [47]:
import sys
 
 
print("User Current Version:-", sys.version)

User Current Version:- 3.9.7 (tags/v3.9.7:1016ef3, Aug 30 2021, 20:19:38) [MSC v.1929 64 bit (AMD64)]


In [48]:
# clearing temp files 
dir = 'D:\1111\major project\temp'
try:
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))
except:
    print('files in use')

files in use
